# Classes Zanetti 

- Identificar padrão nas classes identificadas por Zanetti (2019)

In [1]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from skimage import filters, io, morphology, segmentation
from skimage.color import rgb2gray
from skimage.measure import label, regionprops

plt.interactive(True)


def open_image(fname):
    return io.imread(fname)


def image_segmentation(img):
    gray = rgb2gray(img)
    mask = gray > filters.threshold_otsu(gray)
    borders = segmentation.clear_border(mask).astype(np.int64)
    segmentation.mark_boundaries(gray, borders)
    label_img = label(borders)
    regions = regionprops(label_img)
    return mask, borders, regions


def find_pellet(regions):
    area = 0
    for region in regions:
        if region.area > area:
            area = region.area
            pellet = region
    return pellet


def mask_background(mask, pellet):
    threshold = pellet.area * 0.01
    return morphology.remove_small_objects(mask, pellet.area - threshold)


def show_masked(img, borders, maskr):
    fig, (ax0, ax1) = plt.subplots(ncols=2, figsize=(8, 8))

    masked = img.copy()
    masked[~maskr] = 255
    ax0.imshow(masked)
    ax1.imshow(img)
    ax1.contour(borders, [0.5], colors="r")

In [2]:
from glob import glob

fnames = glob("class-zanetti/*.jpg")

for fname in fnames:
    pellet = Path(fname).stem

    img = open_image(fname)

    mask, borders, regions = image_segmentation(img)
    area = find_pellet(regions)

    maskr = mask_background(mask, area)

    print(f"Processing image {fname}.")
    savename = f"masks-zanetti/{pellet}"
    np.save(savename, maskr)
    print(f"Saved mask into {savename}.\n")

Processing image class-zanetti/alto_cl1_p2_joaq_dunaiii_h0005.jpg.
Saved mask into masks-zanetti/alto_cl1_p2_joaq_dunaiii_h0005.

Processing image class-zanetti/muitoalto_cl1_p4_brava_deixa5_f0010.jpg.
Saved mask into masks-zanetti/muitoalto_cl1_p4_brava_deixa5_f0010.

Processing image class-zanetti/baixo_cl1_p2_joaq_dunaiii_h0002.jpg.
Saved mask into masks-zanetti/baixo_cl1_p2_joaq_dunaiii_h0002.

Processing image class-zanetti/nulo_cl1_p2_joaq_dunaiii_h0006.jpg.
Saved mask into masks-zanetti/nulo_cl1_p2_joaq_dunaiii_h0006.

Processing image class-zanetti/baixo_cl1_p6_moca2_deixa3_a0006.jpg.
Saved mask into masks-zanetti/baixo_cl1_p6_moca2_deixa3_a0006.

Processing image class-zanetti/nulo_cl1_p2_joaq_dunaiii_g0006.jpg.
Saved mask into masks-zanetti/nulo_cl1_p2_joaq_dunaiii_g0006.

Processing image class-zanetti/muitoalto_cl1_p4_brava_deixa5_f0012.jpg.
Saved mask into masks-zanetti/muitoalto_cl1_p4_brava_deixa5_f0012.

Processing image class-zanetti/moderado_cl1_p2_joaq_dunai_c0009.jp

In [3]:
from glob import glob
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
from skimage import io


def get_dominant_color(pil_img, palette_size=16):
    # Resize image to speed up processing
    img = pil_img.copy()
    img.thumbnail((100, 100))

    # Reduce colors (uses k-means internally)
    paletted = img.convert("P", palette=Image.ADAPTIVE, colors=palette_size)

    # Find the color that occurs most often
    palette = paletted.getpalette()
    color_counts = sorted(paletted.getcolors(), reverse=True)
    idx = 1  # We want the second one to avoid getting the mask, otherwise use 0.
    palette_index = color_counts[idx][
        1
    ]  # We want the second one to avoid getting the mask
    dominant_color = palette[palette_index * 3 : palette_index * 3 + 3]

    return dominant_color

In [5]:
fnames = glob("masks-zanetti/*.npy")


dataset = {}
for fname in fnames:
    pellet = Path(fname).stem
    img = io.imread(f"class-zanetti/{pellet}.jpg")
    maskr = np.load(f"masks-zanetti/{pellet}.npy")

    fig, (ax0, ax1, ax2) = plt.subplots(ncols=3, figsize=(15, 5))
    fig.suptitle(pellet)

    ax0.imshow(img)

    # Remove background
    masked = img.copy()
    masked[~maskr] = 255
    ax1.imshow(masked)

    # Dominant color
    pil_img = Image.fromarray(masked)

    R, G, B = get_dominant_color(pil_img)
    dataset.update({pellet: (R, G, B)})

    colored = masked.copy()
    colored[maskr, 0] = R
    colored[maskr, 1] = G
    colored[maskr, 2] = B
    ax2.imshow(colored)

    saved = f"temp-zanetti/check-{pellet}.png"
    print(f"Saving check image {saved}.")
    fig.savefig(saved)
    plt.close()

Saving check image temp-zanetti/check-alto_cl1_p4_brava_deixa4_d0009.png.
Saving check image temp-zanetti/check-muitoalto_cl1_p4_brava_deixa5_f0012.png.
Saving check image temp-zanetti/check-muitoalto_cl1_p4_brava_deixa5_f0010.png.
Saving check image temp-zanetti/check-moderado_cl1_p2_joaq_dunaiii_g0008.png.
Saving check image temp-zanetti/check-alto_cl1_p1_psul_deixa1_g0005.png.
Saving check image temp-zanetti/check-nulo_cl1_p3_brava_deixa3_h0008.png.
Saving check image temp-zanetti/check-alto_cl1_p2_joaq_dunaiii_h0005.png.
Saving check image temp-zanetti/check-baixo_cl1_p6_moca2_deixa3_a0006.png.
Saving check image temp-zanetti/check-baixo_cl1_p2_joaq_dunaiii_h0002.png.
Saving check image temp-zanetti/check-moderado_cl1_p2_joaq_dunaiii_h0003.png.
Saving check image temp-zanetti/check-moderado_cl1_p2_joaq_dunai_c0009.png.
Saving check image temp-zanetti/check-nulo_cl1_p2_joaq_dunaiii_g0006.png.
Saving check image temp-zanetti/check-nulo_cl1_p2_joaq_dunaiii_h0006.png.
Saving check imag

In [6]:
# Save final data
df = pd.DataFrame(dataset, index=("R", "G", "B")).T
df = df.sort_index(ascending = True)
df.to_csv("RGB_values_classeszanetti.csv")
df

,R,G,B
alto_cl1_p1_psul_deixa1_g0005,152,116,3
alto_cl1_p2_joaq_dunaiii_h0005,194,167,2
alto_cl1_p4_brava_deixa4_d0009,186,167,6
baixo_cl1_p2_joaq_dunaiii_h0002,178,170,114
baixo_cl1_p6_moca2_deixa3_a0005,174,191,162
baixo_cl1_p6_moca2_deixa3_a0006,186,193,155
baixo_cl1_p9_moca2_deixa5_b0001,193,192,198
moderado_cl1_p2_joaq_dunai_c0009,199,187,90
moderado_cl1_p2_joaq_dunaiii_g0008,146,126,10
moderado_cl1_p2_joaq_dunaiii_h0003,184,168,64


In [26]:
df["G/B"] = round((df['G'] / df['B']),3)
df["R/G"] = round((df['R'] / df['G']),3)
df["R/B"] = round((df['R'] / df['B']),3)
df

,R,G,B,G/B,R/G,R/B
alto_cl1_p1_psul_deixa1_g0005,152,116,3,38.667,1.310,50.667
alto_cl1_p2_joaq_dunaiii_h0005,194,167,2,83.500,1.162,97.000
alto_cl1_p4_brava_deixa4_d0009,186,167,6,27.833,1.114,31.000
baixo_cl1_p2_joaq_dunaiii_h0002,178,170,114,1.491,1.047,1.561
baixo_cl1_p6_moca2_deixa3_a0005,174,191,162,1.179,0.911,1.074
baixo_cl1_p6_moca2_deixa3_a0006,186,193,155,1.245,0.964,1.200
baixo_cl1_p9_moca2_deixa5_b0001,193,192,198,0.970,1.005,0.975
moderado_cl1_p2_joaq_dunai_c0009,199,187,90,2.078,1.064,2.211
moderado_cl1_p2_joaq_dunaiii_g0008,146,126,10,12.600,1.159,14.600
moderado_cl1_p2_joaq_dunaiii_h0003,184,168,64,2.625,1.095,2.875


In [10]:
nulo = df.iloc[13:18]
nulo.describe()

,R,G,B
count,4.000000,4.000000,4.000000
mean,150.250000,159.500000,161.500000
std,28.756159,37.775654,64.272856
min,114.000000,119.000000,106.000000
25%,139.500000,142.250000,129.250000
50%,151.500000,154.500000,143.000000
75%,162.250000,171.750000,175.250000
max,184.000000,210.000000,254.000000


In [21]:
#DADOS NULO (MÉDIA E DESVIO PADRAO)
mednuloR = 150.25
stdnuloR = 28.76
mednuloG = 159.50
stdnuloG = 37.78
mednuloB = 161.50
stdnuloB = 64.27

In [16]:
baixo = df.iloc[3:7]
baixo.describe()

,R,G,B
count,4.000000,4.000000,4.00000
mean,182.750000,186.500000,157.25000
std,8.460693,11.030261,34.44198
min,174.000000,170.000000,114.00000
25%,177.000000,185.750000,144.75000
50%,182.000000,191.500000,158.50000
75%,187.750000,192.250000,171.00000
max,193.000000,193.000000,198.00000


In [20]:
#DADOS BAIXO (MÉDIA E DESVIO PADRAO)
medbaixoR = 182.75
stdbaixoR = 8.46
medbaixoG = 186.50
stdbaixoG = 11.03
medbaixoB = 157.25
stdbaixoB = 34.44

In [17]:
moderado = df.iloc[7:10]
moderado.describe()

,R,G,B
count,3.000000,3.000000,3.000000
mean,176.333333,160.333333,54.666667
std,27.319102,31.214313,40.808496
min,146.000000,126.000000,10.000000
25%,165.000000,147.000000,37.000000
50%,184.000000,168.000000,64.000000
75%,191.500000,177.500000,77.000000
max,199.000000,187.000000,90.000000


In [22]:
#DADOS MODERADO (MÉDIA E DESVIO PADRAO)
medmodR = 176.33
stdmodR = 27.32
medmodG = 160.33
stdmodG = 31.21
medmodB = 54.67
stdmodB = 40.81

In [13]:
alto = df.iloc[0:3]
alto.describe()

,R,G,B
count,3.000000,3.000000,3.000000
mean,177.333333,150.000000,3.666667
std,22.300972,29.444864,2.081666
min,152.000000,116.000000,2.000000
25%,169.000000,141.500000,2.500000
50%,186.000000,167.000000,3.000000
75%,190.000000,167.000000,4.500000
max,194.000000,167.000000,6.000000


In [23]:
#DADOS ALTO (MÉDIA E DESVIO PADRAO)
medaltoR = 177.33
stdaltoR = 22.30
medaltoG = 150.00
stdaltoG = 29.44
medaltoB = 3.67
stdaltoB = 2.08

In [18]:
muitoalto = df.iloc[10:13]
muitoalto.describe()

,R,G,B
count,3.000000,3.000000,3.000000
mean,139.333333,92.333333,5.666667
std,14.468356,22.052967,3.785939
min,130.000000,68.000000,3.000000
25%,131.000000,83.000000,3.500000
50%,132.000000,98.000000,4.000000
75%,144.000000,104.500000,7.000000
max,156.000000,111.000000,10.000000


In [24]:
#DADOS MUITO ALTO (MÉDIA E DESVIO PADRAO)
medmuitoaltoR = 139.33
stdmuitoaltoR = 14.47
medmuitoaltoG = 92.33
stdmuitoaltoG = 22.05
medmuitoaltoB = 5.67
stdmuitoaltoB = 3.78

In [25]:
# TABELA MÉDIAS E DESVIO PADRÃO RGB PARA AMARELAMENTO
import pandas as pd

index = ('nulo', 'baixo', 'moderado', 'alto', 'muito alto')
data = {'Média R':[mednuloR,medbaixoR,medmodR,medaltoR,medmuitoaltoR],
        'STD R':[stdnuloR,stdbaixoR,stdmodR,stdaltoR,stdmuitoaltoR],
        'Média G':[mednuloG,medbaixoG,medmodG,medaltoG,medmuitoaltoG],
        'STD G':[stdnuloG,stdbaixoG,stdmodG,stdaltoG,stdmuitoaltoG],
        'Média B':[mednuloB,medbaixoB,medmodB,medaltoB,medmuitoaltoB],
        'STD B':[stdnuloB,stdbaixoB,stdmodB,stdaltoB,stdmuitoaltoB]
       }

dfcrit = pd.DataFrame(data, index).T
dfcrit

,nulo,baixo,moderado,alto,muito alto
Média R,150.25,182.75,176.33,177.33,139.33
STD R,28.76,8.46,27.32,22.30,14.47
Média G,159.50,186.50,160.33,150.00,92.33
STD G,37.78,11.03,31.21,29.44,22.05
Média B,161.50,157.25,54.67,3.67,5.67
STD B,64.27,34.44,40.81,2.08,3.78


## Critério de Avaliação 

- Removi a fotografia 

> "alto_cl1_p2_joaq_limsup2_b0004.jpg" - outlier nos dados 

- A) Utilizando as Médias:

> Nulo: **Banda B** => 160 e 1.5 =< **Razão G/B** > 0 

> Baixo: 160 > **Banda B** => 55

> Moderado: 55 > **Banda B** => 10

> Alto: **Banda B** =< 10 e **Banda G** >= 115

> Muito Alto: **Banda B** =< 10 e **Banda G** <= 115

- B) Talvez pro nulo utilizar a razão G/B próxima de 1